In [1]:
# Data visualization
import numpy as np
import pandas as pd 

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
import keras_tuner as kt

# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

import pickle

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Determine important landmarks for barbell row
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_ELBOW",
    "RIGHT_ELBOW",
    "LEFT_WRIST",
    "RIGHT_WRIST",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE",
]

# Generate all columns of the data frame
HEADERS = ["label"]  # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [3]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train.csv", sep=',', encoding='utf-8', index=False)


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))


df01 = describe_dataset("./torso_train.csv")

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v']
Number of rows: 6938 
Number of columns: 53

Labels: 
torso_correct    6255
torso_error       683
Name: label, dtype: int64

Missing values: False

D

In [4]:
# load dataset
df = describe_dataset("./torso_train.csv")
# Categorizing label
df.loc[df["label"] == "torso_correct", "label"] = 0
df.loc[df["label"] == "torso_error", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v']
Number of rows: 6938 
Number of columns: 53

Labels: 
torso_correct    6255
torso_error       683
Name: label, dtype: int64

Missing values: False

D

In [5]:
# Standard Scaling of features
# Dump input scaler
with open("./model/input_scaler_torso.pkl", "rb") as f2:
    sc = pickle.load(f2)   

x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2, random_state=1234)

In [7]:
stop_early = EarlyStopping(monitor='loss', patience=3)

# Final Results
final_models = {}

In [8]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")
            

def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer", "epoch"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

In [9]:
def model_3l_builder(hp):
    model = Sequential()
    model.add(Dense(52, input_dim = 52, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [10]:
tuner_3l_torso = kt.Hyperband(
    model_3l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_7'
)
tuner_3l_torso.search(x_train, y_train, epochs=10, callbacks=[stop_early])

Trial 26 Complete [00h 00m 06s]
accuracy: 0.9590991139411926

Best accuracy So Far: 0.9724324345588684
Total elapsed time: 00h 01m 02s
INFO:tensorflow:Oracle triggered exit


In [11]:
model_3l = get_best_model(tuner_3l_torso)
model_3l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x0000015F64858940>
Layer-2: 352 units, func:  <function relu at 0x0000015F64858940>
Layer-3: 2 units, func:  <function softmax at 0x0000015F64850EE0>

Other params:
learning_rate: 0.001
Epoch 1/100
555/555 [==============================] - 2s 3ms/step - loss: 0.2103 - accuracy: 0.9137 - val_loss: 0.1642 - val_accuracy: 0.9344
Epoch 2/100
555/555 [==============================] - 2s 4ms/step - loss: 0.1327 - accuracy: 0.9418 - val_loss: 0.1353 - val_accuracy: 0.9388
Epoch 3/100
555/555 [==============================] - 2s 3ms/step - loss: 0.1216 - accuracy: 0.9508 - val_loss: 0.1227 - val_accuracy: 0.9474
Epoch 4/100
555/555 [==============================] - 2s 3ms/step - loss: 0.0979 - accuracy: 0.9613 - val_loss: 0.1368 - val_accuracy: 0.9438
Epoch 5/100
555/555 [==============================] - 2s 3ms/step - loss: 0.0879 - accuracy: 0.9641 - val_loss: 0.1295 - val_accuracy: 0.9503
Epoch 6/100
555/555 [====

In [12]:
final_models["3_layers_torso"] = model_3l

In [13]:
def model_5l_builder(hp):
    model = Sequential()
    model.add(Dense(52, input_dim = 52, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [15]:
import keras
tuner_5l_torso = kt.Hyperband(
    model_5l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_8'
)

tuner_5l_torso.search(
    x_train, 
    y_train, 
    epochs=10, 
    callbacks=[keras.callbacks.EarlyStopping(patience=5)]
)

Trial 30 Complete [00h 00m 10s]
accuracy: 0.9810810685157776

Best accuracy So Far: 0.9834234118461609
Total elapsed time: 00h 02m 11s
INFO:tensorflow:Oracle triggered exit


In [16]:
model_5l = get_best_model(tuner_5l_torso)
model_5l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x0000015F64858940>
Layer-2: 288 units, func:  <function relu at 0x0000015F64858940>
Layer-3: 288 units, func:  <function relu at 0x0000015F64858940>
Layer-4: 224 units, func:  <function relu at 0x0000015F64858940>
Layer-5: 2 units, func:  <function softmax at 0x0000015F64850EE0>

Other params:
learning_rate: 0.001
Epoch 1/100
555/555 [==============================] - 3s 4ms/step - loss: 0.2091 - accuracy: 0.9139 - val_loss: 0.1495 - val_accuracy: 0.9301
Epoch 2/100
555/555 [==============================] - 2s 4ms/step - loss: 0.1381 - accuracy: 0.9389 - val_loss: 0.1366 - val_accuracy: 0.9395
Epoch 3/100
555/555 [==============================] - 2s 3ms/step - loss: 0.1151 - accuracy: 0.9481 - val_loss: 0.1348 - val_accuracy: 0.9460
Epoch 4/100
555/555 [==============================] - 2s 3ms/step - loss: 0.1029 - accuracy: 0.9596 - val_loss: 0.1429 - val_accuracy: 0.9445
Epoch 5/100
555/555 [=================

In [17]:
final_models["5_layers_torso"] = model_5l


In [18]:
def model_7lD_builder(hp):
    model = Sequential()
    model.add(Dense(52, input_dim = 52, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_dropout_1 = hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)
    hp_dropout_2 = hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_1))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_2))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [20]:
tuner_7lD_torso = kt.Hyperband(
    model_7lD_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_9'
)
tuner_7lD_torso.search(x_train, y_train, epochs=10, callbacks=[keras.callbacks.EarlyStopping(patience=5)])

Trial 30 Complete [00h 00m 10s]
accuracy: 0.9363964200019836

Best accuracy So Far: 0.9726126194000244
Total elapsed time: 00h 02m 17s
INFO:tensorflow:Oracle triggered exit


In [21]:
model_7lD = get_best_model(tuner_7lD_torso)
model_7lD.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x0000015F64858940>
Layer-2: 96 units, func:  <function relu at 0x0000015F64858940>
Layer-3: 0 units, func: None
Layer-4: 320 units, func:  <function relu at 0x0000015F64858940>
Layer-5: 0 units, func: None
Layer-6: 256 units, func:  <function relu at 0x0000015F64858940>
Layer-7: 2 units, func:  <function softmax at 0x0000015F64850EE0>

Other params:
dropout_1: 0.2
dropout_2: 0.5
learning_rate: 0.001
Epoch 1/100
555/555 [==============================] - 3s 4ms/step - loss: 0.2252 - accuracy: 0.9137 - val_loss: 0.1541 - val_accuracy: 0.9316
Epoch 2/100
555/555 [==============================] - 2s 3ms/step - loss: 0.1615 - accuracy: 0.9368 - val_loss: 0.1603 - val_accuracy: 0.9316
Epoch 3/100
555/555 [==============================] - 2s 4ms/step - loss: 0.1307 - accuracy: 0.9465 - val_loss: 0.1761 - val_accuracy: 0.9251
Epoch 4/100
555/555 [==============================] - 2s 4ms/step - loss: 0.1297 - accuracy: 

In [22]:
final_models["7_layers_with_dropout_torso"] = model_7lD

In [23]:
def model_7l_builder(hp):
    model = Sequential()
    model.add(Dense(52, input_dim = 52, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_layer_4 = hp.Int('layer_4', min_value=32, max_value=512, step=32)
    hp_layer_5 = hp.Int('layer_5', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(units=hp_layer_4, activation=hp_activation))
    model.add(Dense(units=hp_layer_5, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [24]:
tuner_7l_torso = kt.Hyperband(
    model_7l_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_10'
)
tuner_7l_torso.search(x_train, y_train, epochs=10, callbacks=[keras.callbacks.EarlyStopping(patience=5)])

Trial 30 Complete [00h 00m 07s]
accuracy: 0.9598197937011719

Best accuracy So Far: 0.9814414381980896
Total elapsed time: 00h 02m 28s
INFO:tensorflow:Oracle triggered exit


In [25]:
model_7l = get_best_model(tuner_7l_torso)
model_7l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x0000015F64858940>
Layer-2: 224 units, func:  <function relu at 0x0000015F64858940>
Layer-3: 416 units, func:  <function relu at 0x0000015F64858940>
Layer-4: 288 units, func:  <function relu at 0x0000015F64858940>
Layer-5: 512 units, func:  <function relu at 0x0000015F64858940>
Layer-6: 128 units, func:  <function relu at 0x0000015F64858940>
Layer-7: 2 units, func:  <function softmax at 0x0000015F64850EE0>

Other params:
learning_rate: 0.0001
Epoch 1/100
555/555 [==============================] - 4s 7ms/step - loss: 0.2417 - accuracy: 0.9094 - val_loss: 0.1660 - val_accuracy: 0.9258
Epoch 2/100
555/555 [==============================] - 3s 5ms/step - loss: 0.1509 - accuracy: 0.9328 - val_loss: 0.1462 - val_accuracy: 0.9359
Epoch 3/100
555/555 [==============================] - 3s 5ms/step - loss: 0.1206 - accuracy: 0.9485 - val_loss: 0.1493 - val_accuracy: 0.9395
Epoch 4/100
555/555 [=============================

In [26]:
final_models["7_layers_torso"] = model_7l

In [27]:
for name, model in final_models.items():
    print(f"{name}: ", end="")
    describe_model(model)
    print()

3_layers_torso: Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x0000015F64858940>
Layer-2: 352 units, func:  <function relu at 0x0000015F64858940>
Layer-3: 2 units, func:  <function softmax at 0x0000015F64850EE0>

5_layers_torso: Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x0000015F64858940>
Layer-2: 288 units, func:  <function relu at 0x0000015F64858940>
Layer-3: 288 units, func:  <function relu at 0x0000015F64858940>
Layer-4: 224 units, func:  <function relu at 0x0000015F64858940>
Layer-5: 2 units, func:  <function softmax at 0x0000015F64850EE0>

7_layers_with_dropout_torso: Describe models architecture
Layer-1: 52 units, func:  <function relu at 0x0000015F64858940>
Layer-2: 96 units, func:  <function relu at 0x0000015F64858940>
Layer-3: 0 units, func: None
Layer-4: 320 units, func:  <function relu at 0x0000015F64858940>
Layer-5: 0 units, func: None
Layer-6: 256 units, func:  <function relu at 0x0000015F64858940>
Layer-7: 2 uni

In [28]:
train_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(x_test, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])
    
    train_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

train_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(train_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,7_layers_with_dropout_torso,"[0.977, 0.783]","[0.979, 0.764]","[0.978, 0.774]","[[1239, 26], [29, 94]]"
1,7_layers_torso,"[0.971, 0.827]","[0.986, 0.699]","[0.978, 0.758]","[[1247, 18], [37, 86]]"
2,3_layers_torso,"[0.975, 0.765]","[0.978, 0.74]","[0.976, 0.752]","[[1237, 28], [32, 91]]"
3,5_layers_torso,"[0.968, 0.812]","[0.985, 0.667]","[0.976, 0.732]","[[1246, 19], [41, 82]]"


In [29]:
# load dataset
test_df = describe_dataset("./torso_test.csv")

# Categorizing label
test_df.loc[test_df["label"] == "torso_correct", "label"] = 0
test_df.loc[test_df["label"] == "torso_error", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v']
Number of rows: 2449 
Number of columns: 53

Labels: 
torso_correct    2229
torso_error       220
Name: label, dtype: int64

Missing values: False

D

In [30]:
# Standard Scaling of features
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(sc.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [31]:
test_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(test_x, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(test_y_cat, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])
    
    test_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

test_set_results.sort(key=lambda k: k[1] + k[2] + k[3], reverse=True)
pd.DataFrame(test_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,3_layers_torso,"[0.96, 0.655]","[0.969, 0.586]","[0.965, 0.619]","[[2161, 68], [91, 129]]"
1,7_layers_with_dropout_torso,"[0.954, 0.685]","[0.976, 0.523]","[0.965, 0.593]","[[2176, 53], [105, 115]]"
2,5_layers_torso,"[0.942, 0.672]","[0.981, 0.391]","[0.961, 0.494]","[[2187, 42], [134, 86]]"
3,7_layers_torso,"[0.942, 0.647]","[0.979, 0.391]","[0.96, 0.487]","[[2182, 47], [134, 86]]"


In [32]:
final_models["3_layers_torso"].save("./model/barbell_torso_dp.h5")

In [33]:
for model_name, model in final_models.items():
    model.save(f"./model/{model_name}.h5")